## Part 1 - Produce Data
We elected to use the datagen connector to generate fake data for this assignment. The topic we used was 'stocktrades. The steps were as follows:
*  Open a browser and go to http://localhost:9021/
*  Select the available cluster
*  On the menu bar, select Connect
*  Click on the connect-default cluster in the Connect Clusters list.
*  Click on Add connector
*  Select DatagenConnector
*  Enter connector_stock_trades in the Name field

Then:
Generate a data stream with following configurations:
```
Key converter class: org.apache.kafka.connect.storage.StringConverter
kafka.topic: stocktrades
max.interval: 100
quickstart: Stock_Trades
```


## Part 2 - Using Ksql to create at least 2 streams with filtering from topics

To begin, you need to create a stream called stocktrades with no filtering in place.

In [ ]:
CREATE STREAM STOCKTRADES
   (SIDE STRING, QUANTITY INTEGER, SYMBOL STRING, PRICE INTEGER, ACCOUNT STRING, USERID STRING)
       WITH (KAFKA_TOPIC='stocktrades', VALUE_FORMAT='AVRO');

### Stream 1 - Sell Stream
It may be in the interest of the business to view only streams where the stock was sold and not bought. This would be useful in identifying which shares should be taken as a 'short' position

In [ ]:
CREATE STREAM SELL_TRADES WITH (KAFKA_TOPIC='SELL_TRADES', PARTITIONS=1, REPLICAS=1) AS SELECT
  STOCKTRADES.QUANTITY QUANTITY,
  STOCKTRADES.SYMBOL SYMBOL,
  STOCKTRADES.PRICE PRICE,
  STOCKTRADES.ACCOUNT ACCOUNT,
  STOCKTRADES.USERID USERID
FROM STOCKTRADES STOCKTRADES
WHERE (STOCKTRADES.SIDE = 'SELL')
EMIT CHANGES;


### Stream 2 - Buy Stream
It may also be interesting to the business to see trades that were large buys.

In [ ]:
CREATE STREAM BUY_TRADES WITH (KAFKA_TOPIC='BUY_TRADES', PARTITIONS=1, REPLICAS=1) AS SELECT
  STOCKTRADES.QUANTITY QUANTITY,
  STOCKTRADES.SYMBOL SYMBOL,
  STOCKTRADES.PRICE PRICE,
  STOCKTRADES.ACCOUNT ACCOUNT,
  STOCKTRADES.USERID USERID
FROM STOCKTRADES STOCKTRADES
WHERE (STOCKTRADES.SIDE = 'BUY')
EMIT CHANGES;

### Table 1 - Aggregated Buy Trades


In [ ]:
CREATE TABLE AGG_BUY_ORDERS WITH (KAFKA_TOPIC='AGG_BUY_ORDERS', PARTITIONS=1, REPLICAS=1) AS SELECT
  BUY_TRADES.SYMBOL SYMBOL,
  SUM(BUY_TRADES.QUANTITY) QUANTITY_AGG,
  AVG(BUY_TRADES.PRICE) PRICE_AVG,
  SUM((BUY_TRADES.QUANTITY * BUY_TRADES.PRICE)) VALUE_TRADED
FROM BUY_TRADES BUY_TRADES
WINDOW TUMBLING ( SIZE 60 SECONDS )
GROUP BY BUY_TRADES.SYMBOL
EMIT CHANGES;


### Table 2 - Aggregated Sell Trades

In [ ]:
CREATE TABLE AGG_SELL_ORDERS WITH (KAFKA_TOPIC='AGG_SELL_ORDERS', PARTITIONS=1, REPLICAS=1) AS SELECT
  SELL_TRADES.SYMBOL SYMBOL,
  SUM(SELL_TRADES.QUANTITY) QUANTITY_AGG,
  AVG(SELL_TRADES.PRICE) PRICE_AVG,
  SUM((SELL_TRADES.QUANTITY * SELL_TRADES.PRICE)) VALUE_TRADED
FROM SELL_TRADES SELL_TRADES
WINDOW TUMBLING ( SIZE 60 SECONDS )
GROUP BY SELL_TRADES.SYMBOL
EMIT CHANGES;

## Part 3 - Consume/Transform data with Spark Streaming

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
        .appName('kafka') \
        .getOrCreate()

In [3]:
spark.version

'3.1.1'

In [4]:
spark._jvm.org.apache.hadoop.util.VersionInfo.getVersion()

'3.2.0'

## Raw Data Streams

In [5]:
stream_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "broker:29092") \
  .option("startingOffsets", "earliest") \
  .option("subscribe", "stocktrades") \
  .load()

In [6]:
stream_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
raw_stream = stream_df \
    .writeStream \
    .format("memory") \
    .queryName("raw_stocktrades_view") \
    .start()